# Expose Iceberg - Uniform
## Read Unity Catalog Tables in Snowflake

[How to Read Unity Catalog Tables in Snowflake, in 4 Easy Steps](https://www.databricks.com/blog/read-unity-catalog-tables-in-snowflake)

There are 4 steps to creating a REST catalog integration in Snowflake:

- Enable UniForm on a Delta Lake table in Databricks to make it accessible through the Iceberg REST Catalog
- Register Unity Catalog in Snowflake as your catalog
- Register an S3 Bucket in Snowflake so it recognizes the source data
- Create an Iceberg table in Snowflake so you can query your data

1. With UC as delta catalog 
2. With UC + iceberg-rest endpoint

## With Unity Catalog 

- UC required
- Enable column mapping
- minReaderVersion >= 2 and minWriterVersion >= 7. 
- DBR > 14.3LTS
- Disable DV

In [0]:
%python
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [0]:
use catalog ${catalog};
use ${schema}

In [0]:
create or replace table  uniform_demo_table (id INT, name STRING)
tblproperties (
  'delta.columnMapping.mode' = 'name',
  'delta.enableIcebergCompatV2' = 'true',
  'delta.universalFormat.enabledFormats' = 'iceberg'
);

In [0]:
ALTER TABLE uniform_demo_table
SET TBLPROPERTIES (
  'delta.columnMapping.mode' = 'name',
  'delta.enableIcebergCompatV2' = 'true',
  'delta.universalFormat.enabledFormats' = 'iceberg'
);

In [0]:
%python
for i in range(1, 20):
    spark.sql(f"INSERT INTO uniform_demo_table VALUES ({i}, 'Name_{i}')")


In [0]:
describe detail uniform_demo_table

In [0]:
%python
dbutils.fs.ls("s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/3f3a5dc1-a1c7-40c1-93bd-25f40562265e/_delta_log")


In [0]:
describe extended uniform_demo_table

In [0]:
-- s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/3f3a5dc1-a1c7-40c1-93bd-25f40562265e/metadata/00058-1229fe63-b58b-4a73-9543-9320e404a62c.metadata.json

In [0]:
%python
dbutils.fs.ls("s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/3f3a5dc1-a1c7-40c1-93bd-25f40562265e/metadata/")

In [0]:
%python
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [0]:
VACUUM uniform_demo_table RETAIN 0 HOURS FULL

Iceberg metadata generation is done asynchronously. You can manually trigger Iceberg metadata conversion


In [0]:
MSCK REPAIR TABLE uniform_demo_table SYNC METADATA

## Hive table

In [0]:
describe extended delta.`s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/3f3a5dc1-a1c7-40c1-93bd-25f40562265e/`

In [0]:
%python
for i in range(1, 3):
    spark.sql(f"INSERT INTO uniform_demo_table VALUES ({i}, 'Name_{i}')")